In [5]:
import copy
import csv
import os
import pickle
import librosa
import numpy as np
from scipy import signal
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from collections import defaultdict
import random
import pdb


image = []
audio = []
label = []

data_root = '/home/ducca/CMKD/C2KD/cramed_unpair/CREMA-D/'
class_dict = {'NEU':0, 'HAP':1, 'SAD':2, 'FEA':3, 'DIS':4, 'ANG':5}

visual_feature_path = '/home/ducca/CMKD/C2KD/cramed_unpair/CREMA-D/'
audio_feature_path = '/home/ducca/CMKD/C2KD/cramed_unpair/CREMA-D/AudioWAV/'

train_csv = os.path.join(data_root, 'train.csv')
test_csv = os.path.join(data_root, 'test.csv')

# if mode == 'train':
csv_file = train_csv
# else:
#     csv_file = test_csv

with open(csv_file, encoding='UTF-8-sig') as f2:
    csv_reader = csv.reader(f2)
    for item in csv_reader:
        audio_path = os.path.join(audio_feature_path, item[0] + '.wav')
        visual_path = os.path.join(visual_feature_path, 'Image-{:02d}-FPS'.format(1), item[0])

        if os.path.exists(audio_path) and os.path.exists(visual_path):
            image.append(visual_path)
            audio.append(audio_path)
            label.append(class_dict[item[1]])
        else:
            continue

indices_per_class = defaultdict(list)
for idx, lbl in enumerate(label):
    indices_per_class[lbl].append(idx)

transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


image_samples = os.listdir(image[100])
select_index = np.random.choice(len(image_samples), size=1, replace=False)
select_index.sort()
images = torch.zeros((1, 3, 224, 224))
for i in range(1):
    img = Image.open(os.path.join(image[100], image_samples[i])).convert('RGB')
    img = transform(img)
    images[i] = img
    print(images.shape)

images = torch.permute(images, (1,0,2,3))
print(images)

torch.Size([1, 3, 224, 224])
tensor([[[[-0.7308, -0.7308, -0.7308,  ..., -0.6965, -0.6965, -0.6965],
          [-0.7308, -0.7137, -0.7137,  ..., -0.6965, -0.6965, -0.6965],
          [-0.7308, -0.7137, -0.7137,  ..., -0.6965, -0.6965, -0.6965],
          ...,
          [ 0.0056,  0.0569,  0.1426,  ..., -0.2171, -0.2171, -0.2342],
          [-0.0801,  0.0398,  0.1426,  ..., -0.2684, -0.2171, -0.2342],
          [-0.1486, -0.1828, -0.0801,  ..., -0.3027, -0.2856, -0.2513]]],


        [[[ 0.8354,  0.8354,  0.8354,  ...,  0.8704,  0.8704,  0.8704],
          [ 0.8354,  0.8529,  0.8529,  ...,  0.8704,  0.8704,  0.8704],
          [ 0.8354,  0.8529,  0.8529,  ...,  0.8704,  0.8704,  0.8704],
          ...,
          [ 0.1877,  0.2752,  0.3627,  ...,  0.0126,  0.0126, -0.0049],
          [ 0.1352,  0.2577,  0.3627,  ..., -0.0399,  0.0126, -0.0224],
          [ 0.0651,  0.0301,  0.1352,  ..., -0.0924, -0.0749, -0.0399]]],


        [[[-0.7587, -0.7587, -0.7587,  ..., -0.7238, -0.7238, -0.7238